### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read Data

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_heart = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
df_heart.name = 'Heart DF'
df_heart.head()

In [ ]:
df_heart.info()

In [ ]:
df_heart.describe().T

In [ ]:
def get_shape(df):
    df_name = df.name
    print('Shape of {} is {}'.format(df_name,df.shape))

In [ ]:
get_shape(df_heart)

In [ ]:
df_o2Saturation = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
df_o2Saturation.name = 'o2Saturation DF'
df_o2Saturation.head()

In [ ]:
get_shape(df_o2Saturation)

## Find Data type of each column

In [ ]:
# def get_column_dtype(df):
#     dataTypeSeries = df.dtypes
#     print('Data type of each column of Dataframe :\n',dataTypeSeries)
# get_column_dtype(df_heart)

In [ ]:
df_heart.dtypes

### These columsn should not be int but categorical
Sex, cp, fbs, restecg, slp, caa, thall, output

In [ ]:
df_heart['sex'].value_counts()

In [ ]:
df_heart['sex'] = df_heart['sex'].astype('object', copy=False)

In [ ]:
df_heart['cp'].unique()

In [ ]:
df_heart['cp'] = df_heart['cp'].astype('object', copy=False)

In [ ]:
df_heart['fbs'].value_counts()

In [ ]:
df_heart['fbs'] = df_heart['fbs'].astype('object', copy=False)

In [ ]:
df_heart['restecg'].value_counts()

In [ ]:
df_heart['restecg'] = df_heart['restecg'].astype('object', copy=False)

In [ ]:
df_heart['exng'].value_counts()

In [ ]:
df_heart['exng'] = df_heart['exng'].astype('object', copy=False)

In [ ]:
df_heart['slp'].value_counts()

In [ ]:
df_heart['slp'] = df_heart['slp'].astype('object', copy=False)

In [ ]:
df_heart['caa'].value_counts()

In [ ]:
df_heart['caa'] = df_heart['caa'].astype('object', copy=False)

In [ ]:
df_heart['thall'].value_counts()

In [ ]:
df_heart['thall'] = df_heart['thall'].astype('object', copy=False)

In [ ]:
df_heart['output'].value_counts()

In [ ]:
df_heart['output'] = df_heart['output'].astype('object', copy=False)

In [ ]:
df_heart.dtypes

In [ ]:
# categorical columns
categorical_columns = df_heart.dtypes[df_heart.dtypes == np.object]
categorical_columns

In [ ]:
numerical_columns = df_heart.dtypes[df_heart.dtypes != np.object]
numerical_columns

## Check Distribution of Data

### QQ plot

#### if QQ is  inconcusive h then use Kolmogorov Smirnov test
#### Use Lilliefors test if mean and variance is not pre defined by the user
#### Shapiro Wilk test is most powerful test you can use

In [ ]:
import scipy.stats as stats
import pylab

In [ ]:
def plot_qq(df,feature):
    plt.figure(figsize = (10,6))
    plt.subplot(1,2,1)
    df[feature].hist()
    plt.subplot(1,2,2)
    stats.probplot(df[feature], dist= 'norm', plot=pylab)
    plt.show()

In [ ]:
plot_qq(df_heart,'age')

In [ ]:
plot_qq(df_heart,'trtbps')

In [ ]:
plot_qq(df_heart,'chol')

In [ ]:
plot_qq(df_heart,'thalachh')

In [ ]:
plot_qq(df_heart,'oldpeak')

### Not NORMALLY Distributed

Also it has lots if zero values which in not possible so we are going to correct them

In [ ]:
#df_heart['oldpeak_log'] = np.log(df_heart['oldpeak'])

In [ ]:
#plot_qq(df_heart,'oldpeak_log')

In [ ]:
df_heart[df_heart['oldpeak'] == 0].count()[0]

total 99 rows which is a bigchunk of data; almost one third! meh.

In [ ]:
df_heart_oldpeak = df_heart['oldpeak'].copy()

In [ ]:
df_heart_oldpeak.describe()

I think we should take mean and std of the column without the 0 values because 0 clearly is an error during data collection as oldmeak min value is 0.1 (while ignoring zero values)

In [ ]:
df_heart_oldpeak_new = df_heart_oldpeak[df_heart_oldpeak.values != 0]

In [ ]:
df_heart_oldpeak_new.describe()

As you can see mean and std are very different

In [ ]:
def fillNaN_with_unifrand(df,df_new):
    a = df.values
    # m = np.isnan(a) # mask of NaNs
    m = (df == 0) # mask of zero values
    mu, sigma = df_new.mean(), df_new.std()
    df[m] = np.random.normal(mu, sigma, size=m.sum())
    return df

In [ ]:
fillNaN_with_unifrand(df_heart_oldpeak,df_heart_oldpeak_new)

In [ ]:
df_heart['oldpeak_imputed'] = df_heart_oldpeak

In [ ]:
plot_qq(df_heart,'oldpeak_imputed')

NOW it is normally distributed

Now dropping old column and also adding an additional column where oldpeak value was zero

In [ ]:
df_heart['oldpeak_zero'] = 0

In [ ]:
df_heart.oldpeak_zero[df_heart['oldpeak'] == 0] = 1

In [ ]:
df_heart.drop('oldpeak', axis = 1, inplace = True)

### Here we did one mistake thats is DATA LEAKAGE
due to taking mean and std of whole data in oldpeak column
we should have split the data first then try to impute the zero values
#### OOPS!!!

# EDA

In [ ]:
sns.set_theme(style="white")
sns.despine()

In [ ]:
categorical_columns

### Count Plots

In [ ]:
# sns.countplot(y='sex', data=df_heart, hue='output',palette="Set3")
# sns.despine()

In [ ]:
sns.catplot(y="cp", hue="sex", data=df_heart, col = 'output',
            kind = 'count',palette="Set3",
            height=4, aspect= 0.9
           )

In [ ]:
sns.catplot(y="fbs", hue="sex", data=df_heart, col = 'output',
            kind = 'count',palette="Set3",
            height=4, aspect= 0.9
           )

In [ ]:
sns.catplot(y="restecg", hue="sex", data=df_heart, col = 'output',
            kind = 'count',palette="Set3",
            height=4, aspect= 0.9
           )

In [ ]:
sns.catplot(y="exng", hue="sex", data=df_heart, col = 'output',
            kind = 'count',palette="Set3",
            height=4, aspect= 0.9
           )

In [ ]:
sns.catplot(y="slp", hue="sex", data=df_heart, col = 'output',
            kind = 'count',palette="Set3",
            height=4, aspect= 0.9
           )

In [ ]:
sns.catplot(y="caa", hue="sex", data=df_heart, col = 'output',
            kind = 'count',palette="Set3",
            height=4, aspect= 0.9
           )

In [ ]:
sns.catplot(y="thall", hue="sex", data=df_heart, col = 'output',
            kind = 'count',palette="Set3",
            height=4, aspect= 0.9
           )

### Numerical Data

In [ ]:
numerical_columns

In [ ]:
sns.violinplot(data=df_heart['age'])

In [ ]:
sns.distplot(a=df_heart['age'])

In [ ]:
sns.violinplot(data=df_heart['thalachh'], palette="deep")

In [ ]:
sns.distplot(a=df_heart['thalachh'])

In [ ]:
sns.violinplot(data=df_heart['chol'], palette="deep")

In [ ]:
sns.distplot(a=df_heart['chol'])

In [ ]:
sns.violinplot(data=df_heart['trtbps'], palette="deep")

In [ ]:
sns.distplot(a=df_heart['trtbps'])

In [ ]:
sns.violinplot(data=df_heart['oldpeak_imputed'], palette="deep")

In [ ]:
sns.distplot(a=df_heart['oldpeak_imputed'])

## Some Observations

Values for feature caa class 4 is very low; number of major vessels (0-3)

Values for feature thall class 0 is very low;

Values for feature restecg class 0 is very low;

    rest_ecg : resting electrocardiographic results

        Value 0: normal

        Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)

        Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

#### We have outliers in chol, oldpeak_imputed and trtbps
For now we are not making any changes we will move on to feature engineering

## Feature Engineering

#### Lets check correlation via heat map

In [ ]:
df_heart['output'] = df_heart['output'].astype('int')

In [ ]:
sns.heatmap(df_heart.iloc[:, lambda df_heart: df_heart.columns.str.contains('age|trtbps|chol|thalachh|oldpeak_imputed|output',
                                              case=False)].corr(),
            center=0,annot=True, cbar = True
           ) 

### Observations
age and thalachh are negatively correlated but only 40%

output is very much goverened by thalachh and not by chol as we assumed initially

Interesting lets look into thalachh more

In [ ]:
df_heart['thalachh'].describe()

It is a little left skweed but for now not doing anything as it does not vary form normal dist that much and we are applying normalization anyway

In [ ]:
#df_heart['output'] = df_heart['output'].astype('object')
df_heart['output'] = df_heart['output'].astype('int')

### One hot encoding

In [ ]:
categorical_columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
df_heart_onehotencoded = pd.get_dummies(df_heart, columns=['cp', 'restecg', 'slp', 'caa', 'thall'], drop_first = True)

In [ ]:
df_heart_onehotencoded.head()

### Must Split test data to avoid data leakage

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_heart_onehotencoded.loc[:,df_heart_onehotencoded.columns != 'output']
y = df_heart_onehotencoded['output']

In [ ]:
X.head()

In [ ]:
y = pd.DataFrame(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Normalization

In [ ]:
min_max_scaler = MinMaxScaler()

In [ ]:
X_train_scaled = min_max_scaler.fit_transform(X_train)

In [ ]:
X_test_scaled = min_max_scaler.transform(X_test)

In [ ]:
X_train_scaled = pd.DataFrame(X_train_scaled)

In [ ]:
X_test_scaled = pd.DataFrame(X_test_scaled)

In [ ]:
X_train_scaled.head()

## Model Building

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

### Logistic Regression

In [ ]:
log_reg = LogisticRegression()

In [ ]:
score = cross_val_score(log_reg,X_train_scaled,y=y_train, 
                        cv=3, n_jobs=None, scoring = 'accuracy',
                        verbose=0)

In [ ]:
score.mean()

### Random Forest Classifier

In [ ]:
rf = RandomForestClassifier()

In [ ]:
score = cross_val_score(rf,X_train_scaled,y=y_train, 
                        cv=3, n_jobs=None, scoring = 'accuracy',
                        verbose=0)

In [ ]:
score.mean()

### Xgboost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
xgboost = GradientBoostingClassifier()

In [ ]:
score = cross_val_score(xgboost,X_train_scaled,y=y_train, 
                        cv=3, n_jobs=None, scoring = 'accuracy',
                        verbose=0)

In [ ]:
score.mean()

### SVC

In [ ]:
svc = SVC()

In [ ]:
score = cross_val_score(svc,X_train_scaled,y=y_train, 
                        cv=3, n_jobs=None, scoring = 'accuracy',
                        verbose=0)

In [ ]:
score.mean()

#### All models are relatively close with XgBoost as lowest so we will go ahead with this.

### Hypertuning Model

In [ ]:
xgboost.fit(X_train_scaled, y_train)

In [ ]:
pred = xgboost.predict(X_test_scaled)

In [ ]:
confusion_matrix(y_test, pred)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, pred, normalize=False)

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, pred)

In [ ]:
precision[1]

In [ ]:
recall[1]

In [ ]:
thresholds[1]